In [30]:
import pandas as pd
import numpy as np
import re
import difflib
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, naive_bayes, svm
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import logging
from keras.models import Sequential
from keras import layers
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.sequence import pad_sequences

In [31]:
# read in full data
data = pd.read_csv('Full+data.csv')
data.head(1)

,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# reduce to non nan columns
cols = data.columns[:14]
data = data.loc[:,cols]
data.shape

(42373, 14)

In [33]:
# import extra data
extra_data = pd.read_csv('extra_data.csv')
extra_data.head(1)


,product_id,mpn,brand,brand_category,name,labels,updated_at,bc_product_id,saleable,details,created_at,brand_canonical_url,notes,description
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,5529544,A.L.C.,Unknown,Lennox High Waist Cotton & Linen Pants,[],2020-04-17 15:44:57.785000+00:00,5021,2020-04-17 15:44:57.785000+00:00,"True to size. High rise.\n31"" inseam; 14"" leg ...",2020-04-15 21:59:56.695000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,NaN,High-rise trousers tailored from a cool Italia...


In [34]:
data.head(1)

,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,tsv,bc_product_id
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,Leather/synthetic upper\nLace-up closure\nText...,"{""Needs Attributes""}","'design':12 'fila':1A 'fit':3A,6 'leather':7 '...",NaN


In [35]:
extra_data.head(1)

,product_id,mpn,brand,brand_category,name,labels,updated_at,bc_product_id,saleable,details,created_at,brand_canonical_url,notes,description
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,5529544,A.L.C.,Unknown,Lennox High Waist Cotton & Linen Pants,[],2020-04-17 15:44:57.785000+00:00,5021,2020-04-17 15:44:57.785000+00:00,"True to size. High rise.\n31"" inseam; 14"" leg ...",2020-04-15 21:59:56.695000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,NaN,High-rise trousers tailored from a cool Italia...


Extra_data and Full_data do not have the same columns. So, we subset the data for only important columns and the join the dataframes

In [36]:
# columns to use from extra_data
cols = ['product_id', 'brand', 'name', 'description', 'brand_category', 'brand_canonical_url','details']

In [37]:
extra_data = extra_data.loc[:,cols]
data = data.loc[:,cols]

In [38]:
print(extra_data.shape)
data.shape

(6621, 7)


(42373, 7)

In [39]:
# we will join brand and brand_category together to brand_info

#data['brand_info'] = data['brand']+data['brand_category']+ " " +data['brand_canonical_url']
#extra_data['brand_info'] = extra_data['brand']+extra_data['brand_category']+data['brand_canonical_url']

#drop brand and brand category

#data = data.drop(['brand','brand_category','brand_canonical_url'],axis = 1)
#extra_data = extra_data.drop(['brand','brand_category','brand_canonical_url'],axis = 1)

In [40]:
# concatenate data and extra data
full_data = pd.concat([data,extra_data])
len(full_data) == len(data) + len(extra_data)

True

Now, we merge the all tags provided to us and join it with the product information

In [41]:
tags = pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx')
tags.shape

(21925, 4)

In [42]:
add_tags = pd.read_csv('usc_additional_tags.csv')
add_tags.shape

(97420, 4)

In [43]:
all_tags = pd.concat([tags,add_tags])
len(all_tags) == len(tags) + len(add_tags)

True

In [44]:
all_tags.head(1)

,product_id,product_color_id,attribute_name,attribute_value
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,Women


In [45]:
# remove product_color_id
all_tags = all_tags.loc[:,['product_id','attribute_name','attribute_value']]
all_tags.shape

(119345, 3)

subsetting the tags just for attribute_name = color. So, that when we merge back there is no multiple records

In [46]:
colors = all_tags[all_tags['attribute_name'].str.contains(r'color', flags=re.IGNORECASE)]
print(f'All :{colors.shape}')
colors = colors.drop_duplicates()
print(f'Clean:{colors.shape}')

All :(9858, 3)
Clean:(9348, 3)


In [47]:
colors['attribute_value'].unique()

array(['Blacks', 'Pinks', 'Browns', 'Golds', 'Whites', 'Reds', 'Navy',
       'Beiges', 'Blues', 'Greens', 'Silvers', 'Neutrals', 'Yellows',
       'Grays', 'Burgundies', 'Purples', 'Multi', 'Oranges', 'Teal',
       'blues', 'blacks', 'yellows', 'oranges', 'whites', 'multi', 'navy',
       'darkbrowns', 'grays', 'greens', 'pinks', 'beiges', 'golds',
       'silvers', 'lightbrowns', 'reds', 'lightbrown', 'burgundies',
       'purples', 'teal'], dtype=object)

We see that naming is not consistent. So, we clean the data to make it same

In [48]:
colors['attribute_value'] = colors['attribute_value'].str.lower()
colors = colors.drop_duplicates()
print(f'Clean:{colors.shape}')

Clean:(9348, 3)


In [49]:
colors['attribute_value'].unique()

array(['blacks', 'pinks', 'browns', 'golds', 'whites', 'reds', 'navy',
       'beiges', 'blues', 'greens', 'silvers', 'neutrals', 'yellows',
       'grays', 'burgundies', 'purples', 'multi', 'oranges', 'teal',
       'darkbrowns', 'lightbrowns', 'lightbrown'], dtype=object)

In [50]:
# light brown and dark brown should be categorized as brown

In [51]:
colors['attribute_value'] = colors['attribute_value'].str.replace('darkbrowns', 'browns')
colors['attribute_value'] = colors['attribute_value'].str.replace('lightbrowns', 'browns')
colors['attribute_value'] = colors['attribute_value'].str.replace('lightbrown', 'browns')

In [52]:
colors.attribute_value.unique()

array(['blacks', 'pinks', 'browns', 'golds', 'whites', 'reds', 'navy',
       'beiges', 'blues', 'greens', 'silvers', 'neutrals', 'yellows',
       'grays', 'burgundies', 'purples', 'multi', 'oranges', 'teal'],
      dtype=object)

In [53]:
# create dummy variables for each color value
colors['is_black'] = colors['attribute_value'].apply(lambda x: 1 if x == 'blacks' else 0)
colors['is_beige'] = colors['attribute_value'].apply(lambda x: 1 if x == 'beiges' else 0)
colors['is_burgundy'] = colors['attribute_value'].apply(lambda x: 1 if x == 'burgundies' else 0)
colors['is_white'] = colors['attribute_value'].apply(lambda x: 1 if x == 'whites' else 0)
colors['is_gray'] = colors['attribute_value'].apply(lambda x: 1 if x == 'grays' else 0)
colors['is_gold'] = colors['attribute_value'].apply(lambda x: 1 if x == 'golds' else 0)
colors['is_blue'] = colors['attribute_value'].apply(lambda x: 1 if x == 'blues' else 0)
colors['is_neutral'] = colors['attribute_value'].apply(lambda x: 1 if x == 'neutrals' else 0)
colors['is_pink'] = colors['attribute_value'].apply(lambda x: 1 if x == 'pinks' else 0)
colors['is_orange'] = colors['attribute_value'].apply(lambda x: 1 if x == 'oranges' else 0)
colors['is_navy'] = colors['attribute_value'].apply(lambda x: 1 if x == 'navy' else 0)
colors['is_brown'] = colors['attribute_value'].apply(lambda x: 1 if x == 'browns' else 0)
colors['is_red'] = colors['attribute_value'].apply(lambda x: 1 if x == 'reds' else 0)
colors['is_yellow'] = colors['attribute_value'].apply(lambda x: 1 if x == 'yellows' else 0)
colors['is_multi'] = colors['attribute_value'].apply(lambda x: 1 if x == 'multi' else 0)
colors['is_green'] = colors['attribute_value'].apply(lambda x: 1 if x == 'greens' else 0)
colors['is_silver'] = colors['attribute_value'].apply(lambda x: 1 if x == 'silvers' else 0)
colors['is_teal'] = colors['attribute_value'].apply(lambda x: 1 if x == 'teal' else 0)
colors['is_purple'] = colors['attribute_value'].apply(lambda x: 1 if x == 'purples' else 0)

In [54]:
colors.shape

(9348, 22)

In [55]:
# tag each product id with all colors that it's tagged as

black = colors[['product_id','is_black']]
tags_black = black.groupby(['product_id'])['is_black'].max().reset_index()

beige = colors[['product_id','is_beige']]
tags_beige = beige.groupby(['product_id'])['is_beige'].max().reset_index()

burgundy = colors[['product_id','is_burgundy']]
tags_burgundy = burgundy.groupby(['product_id'])['is_burgundy'].max().reset_index()

white = colors[['product_id','is_white']]
tags_white = white.groupby(['product_id'])['is_white'].max().reset_index()

gray = colors[['product_id','is_gray']]
tags_gray = gray.groupby(['product_id'])['is_gray'].max().reset_index()

gold = colors[['product_id','is_gold']]
tags_gold = gold.groupby(['product_id'])['is_gold'].max().reset_index()

blue = colors[['product_id','is_blue']]
tags_blue = blue.groupby(['product_id'])['is_blue'].max().reset_index()

neutral = colors[['product_id','is_neutral']]
tags_neutral = neutral.groupby(['product_id'])['is_neutral'].max().reset_index()

pink = colors[['product_id','is_pink']]
tags_pink = pink.groupby(['product_id'])['is_pink'].max().reset_index()

orange = colors[['product_id','is_orange']]
tags_orange = orange.groupby(['product_id'])['is_orange'].max().reset_index()

navy = colors[['product_id','is_navy']]
tags_navy = navy.groupby(['product_id'])['is_navy'].max().reset_index()

brown = colors[['product_id','is_brown']]
tags_brown = brown.groupby(['product_id'])['is_brown'].max().reset_index()

red = colors[['product_id','is_red']]
tags_red = red.groupby(['product_id'])['is_red'].max().reset_index()

yellow = colors[['product_id','is_yellow']]
tags_yellow = yellow.groupby(['product_id'])['is_yellow'].max().reset_index()

multi = colors[['product_id','is_multi']]
tags_multi = multi.groupby(['product_id'])['is_multi'].max().reset_index()

green = colors[['product_id','is_green']]
tags_green = green.groupby(['product_id'])['is_green'].max().reset_index()

silver = colors[['product_id','is_silver']]
tags_silver = silver.groupby(['product_id'])['is_silver'].max().reset_index()

teal = colors[['product_id','is_teal']]
tags_teal = teal.groupby(['product_id'])['is_teal'].max().reset_index()

purple = colors[['product_id','is_purple']]
tags_purple = purple.groupby(['product_id'])['is_purple'].max().reset_index()

In [56]:
df_list = [tags_black, tags_beige, tags_burgundy, tags_white, tags_gray, tags_gold, tags_blue, tags_neutral, tags_pink, tags_orange, tags_navy, tags_brown, tags_red, tags_yellow, tags_multi, tags_green, tags_silver, tags_teal, tags_purple]
for df_ in df_list:
    tags = pd.merge(tags, df_, on='product_id', how='left')
tags_color_all = tags

In [57]:
tags_color_all.to_csv('color.csv')

# Start Cleaning

In [58]:
# merge full data with tags
data_color = pd.merge(full_data,tags_color_all,on='product_id', how = 'inner')

In [60]:
data_color.fillna('UNKNOWNTOKEN',inplace=True)

In [61]:
data_color.to_csv('color_uncleaned.csv')

In [141]:
# function to remove stopwords
from gensim.parsing.preprocessing import STOPWORDS
stop = set(STOPWORDS)
def remove_stopwords(data_col):
    new_list = []
    a = data_col
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            if word not in stop:
                res_words.append(word)
            sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [142]:
# remove stopwords from each of the 4 columns
data_color['description'] = remove_stopwords(data_color['description'])
data_color['details'] = remove_stopwords(data_color['details'])
data_color['name'] = remove_stopwords(data_color['name'])
data_color['brand'] = remove_stopwords(data_color['brand'])
data_color['brand_category'] = remove_stopwords(data_color['brand_category'])

In [143]:
# function to do regex cleaning
def reg_clean(data,col):
    new_list = []
    for i in range(0,len(data)):
        #special characters 
        a = re.sub(r'[^ a-zA-Z0-9]',' ',data.loc[i,col])
        #new lines
        a = re.sub(r'\n',' ', a)
        #remove multiple spaces by a single space
        a = re.sub(r'\s+',' ',a)
        #timestamp
        a = re.sub(r'\b[0-9]{1,}am|[0-9]{1,}pm|[0-9]{4,}|[0-9]ish|1st|2nd|3rd|[0-9]{1,2}th|31st|[0-9]{1,}min(?:utes)?s?|[0-9]{1,}h(?:ou)?rs?|[0-9]{3,}\b','timestamp',a)
        a = re.sub(r'\b[0-9]{1,}timestamp\b','timestamp',a)
        #any numbers as digit
        a = re.sub(r'\b\d+\b','digit',a)
        #number followed by a variable
        a = re.sub(r'\b\d{1,}[a-z]{0,}[0-9]{0,}','varchar',a)
        #html codes
        a = re.sub(r'<.+?>','html',a)
        a = re.sub(r'https|www|com','html',a)
        # additional stopwords
        a = re.sub(r'\ba\b', '', a, flags = re.IGNORECASE)
        a = re.sub(r'\bthis|the\b', '',a, flags = re.IGNORECASE)
        new_list.append(a)
    return new_list

In [144]:
# perform regex cleaning on each column
data_color['description'] = reg_clean(data_color,'description')
data_color['details'] = reg_clean(data_color,'details')
data_color['brand'] = reg_clean(data_color,'brand')
data_color['name'] = reg_clean(data_color, 'name')
data_color['brand_category'] = reg_clean(data_color,'brand_category')

In [145]:
#words = set(map(lambda word: word.replace("\n", ""), open("20k.txt").readlines()))

In [146]:
# function to perform spell checking
#def spellcheck_document(text):
#    new_tokens = []
#    for token in word_tokenize(text):
#        matches = difflib.get_close_matches(token, words, n=1, cutoff=0.7)
#        if len(matches) == 0 or token.lower() in words:
#            new_tokens.append(token)
#        else:
#            new_tokens.append(matches[0])
#    return " ".join(new_tokens)
#spellcheck_document("He is a craezy perzon")

In [147]:
# attempted to spell check for 3 hours, but it only got 1/3 of the way through 1 column
# opted not to run this since I don't have the processing power to do it quickly

In [148]:
#spell check the whole document apart from product_id we have added unknown_text as well so that it does not return unknown
#cols = data_color.columns[1:5]
#for col in cols:
#    new_list = []
#    for i in range(0,len(data_color)):
#        new_list.append(spellcheck_document(data_color.loc[i,col]))
#        if i%5000 == 0:
#            print(i)
#    data_color.col = new_list
#    print(col)
#data_color.head(5)

In [149]:
# function to lemmatize columns
def lemmatize_sentence(data_col):
    new_list = []
    a = data_col 
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            res_words.append(lemmatizer.lemmatize(word).strip(string.punctuation))
        sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [150]:
# lemmatize each column
data_color['description'] = lemmatize_sentence(data_color['description'])
data_color['details'] = lemmatize_sentence(data_color['details'])
data_color['brand'] = lemmatize_sentence(data_color['brand'])
data_color['name'] = lemmatize_sentence(data_color['name'])
data_color['brand_category'] = lemmatize_sentence(data_color['brand_category'])

In [151]:
data_color.head(1)

,product_id,brand,name,description,brand_category,brand_canonical_url,details,product_color_id,attribute_name,attribute_value,...,is_orange,is_navy,is_brown,is_red,is_yellow,is_multi,is_green,is_silver,is_teal,is_purple
0,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah stretch silk camisole,Beige stretch silk Slips digit silk digit span...,Clothing Tops Tanks Camis,https://www.net-a-porter.com/us/en/product/119...,Fits true size normal size Cut slightly loose ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,neckline,Scoop Neck,...,0,0,0,0,0,0,0,0,0,0


#### Collocation

In [152]:
# function to start collocation
stopwords_coll = set(stopwords.words('english') + [".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-","timestamp","varchar","html","digit"])
filter_stops = lambda w: len(w) < 3 or w in stopwords_coll
def collocation_list(data_col):
    new_list = []
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        res_words = []
        for word in words:
            if word not in stopwords_coll:
                res_words.append(word)
        new_list.append(res_words)
    return(new_list)

In [153]:
cols = ['brand', 'brand_category', 'description', 'name','details']
n_list = []
for col in cols:
    n_list.append(collocation_list(data_color[col]))

In [154]:
# we find that dry clean, machine wash, 'fits true to size', 'made in italy' 
# should be collated together

col_list = [['Dry', 'clean'], ['Machine', 'wash'], ['true', 'size'],['fit', 'true_size'], ['Fits', 'true_size'],  
            ['Made', 'Italy']]
data_col = data_color['details']
new_list = []
clean_list = ['Dry','wash','Italy', 'size']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['details'] = new_list

In [155]:
# we find that wide leg, ankle boot, high waist, high rise 
# should be collated together

col_list = [['wide','leg'], ['ankle', 'boot'], ['High', 'Waist'], ['Wide', 'Leg'], ['High', 'Rise'], 
            ['straight', 'leg'], ['shoulder', 'bag']]
data_col = data_color['name']
new_list = []
clean_list = ['leg', 'boot', 'Waist', 'Leg', 'Rise','bag']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['name'] = new_list

In [156]:
# we find that dry clean, machine wash, made in italy, wide leg, best selling, high waisted
# pencil skirt, high rise, and hand wash should be collocated together

col_list = [['Dry','clean'], ['Machine', 'wash'], ['Made', 'Italy'], ['wide', 'leg'], ['best', 'selling'], 
            ['high', 'waisted'], ['pencil', 'skirt'], ['High', 'Rise'], ['high','rise'], ['Hand','wash']]
data_col = data_color['description']
new_list = []
clean_list = ['clean', 'wash', 'Italy', 'leg', 'selling','waisted', 'skirt','Rise', 'rise']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['description'] = new_list

In [157]:
# obtain top 25 collocations by raw frequency for 'description' column
collocation_finder = BigramCollocationFinder.from_documents(n_list[0])
measures = BigramAssocMeasures()
collocation_finder.apply_word_filter(filter_stops)
collocation_finder.nbest(BigramAssocMeasures.likelihood_ratio, 25)

[('Banana', 'Republic'),
 ('Aleksandre', 'Akhalkatsishvili'),
 ('Isabel', 'Marant'),
 ('Ulla', 'Johnson'),
 ('Nili', 'Lotan'),
 ('Jimmy', 'Choo'),
 ('Veronica', 'Beard'),
 ('Victoria', 'Beckham'),
 ('Baum', 'und'),
 ('und', 'Pferdgarten'),
 ('Gianvito', 'Rossi'),
 ('Sally', 'LaPointe'),
 ('ATM', 'Anthony'),
 ('Anthony', 'Thomas'),
 ('Thomas', 'Melillo'),
 ('New', 'York'),
 ('Eileen', 'Fisher'),
 ('Madeleine', 'Thompson'),
 ('Alexander', 'Wang'),
 ('Saint', 'Laurent'),
 ('Carolina', 'Herrera'),
 ('Brock', 'Collection'),
 ('Elie', 'Tahari'),
 ('TWENTY', 'Montr'),
 ('Cult', 'Gaia')]

In [158]:
# we find that  should be collocated together

col_list = [['New', 'York'], ['Jimmy','Choo'], ['Banana','Republic'], ['Victoria','Beckham']]
data_col = data_color['brand']
new_list = []
clean_list = ['York', 'Choo','Republic', 'Beckham']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['brand'] = new_list

In [159]:
# obtain top 25 collocations by raw frequency for 'description' column
collocation_finder = BigramCollocationFinder.from_documents(n_list[1])
measures = BigramAssocMeasures()
collocation_finder.apply_word_filter(filter_stops)
collocation_finder.nbest(BigramAssocMeasures.likelihood_ratio, 25)

[('woman', 'CLOTHING'),
 ('CLOTHING', 'TOPS'),
 ('shirt', 'top'),
 ('CLOTHING', 'DRESSES'),
 ('Boots', 'Ankle'),
 ('woman', 'SHOES'),
 ('Giftsjsholiday', 'HolidayGiftGuide19'),
 ('HolidayGiftGuide19', 'GiftsForHersakstimestamp'),
 ('Clothing', 'Tops'),
 ('Handbags', 'Handbags'),
 ('Mid', 'Heel'),
 ('CLOTHING', 'PANTS'),
 ('Tops', 'Blouses'),
 ('WomensApparel', 'TopsShirtsBlouses'),
 ('Shoes', 'Boots'),
 ('Bags', 'Shoulder'),
 ('Shoulder', 'Bags'),
 ('ApparelAccessories', 'HandbagsShoes'),
 ('CLOTHING', 'SKIRTS'),
 ('Handbags', 'NewPopular'),
 ('OLD', 'ApparelAccessories'),
 ('SHOES', 'SANDALS'),
 ('PantsShortsJumpsuits', 'Trousers'),
 ('Clothing', 'Knitwear'),
 ('Medium', 'Knit')]

In [160]:
col_list = [['Mid', 'Heel'], ['Shoulder', 'Bags']]
data_col = data_color['brand_category']
new_list = []
clean_list = ['Heel', 'Bags']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['brand_category'] = new_list

In [166]:
data_color['is_black'].unique()

array([0.0, 1.0, 'UNKNOWNTOKEN'], dtype=object)

In [168]:
data_color = data_color[data_color['is_black'] != 'UNKNOWNTOKEN']

In [169]:
# export to CSV to use in next notebook
data_color.to_csv('data_color.csv')

In [170]:
data_color.head()

,product_id,brand,name,description,brand_category,brand_canonical_url,details,product_color_id,attribute_name,attribute_value,...,is_orange,is_navy,is_brown,is_red,is_yellow,is_multi,is_green,is_silver,is_teal,is_purple
0,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah stretch silk camisole,Beige stretch silk Slips digit silk digit span...,Clothing Tops Tanks Camis,https://www.net-a-porter.com/us/en/product/119...,Fits true_size normal Cut slightly loose fit L...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,neckline,Scoop Neck,...,0,0,0,0,0,0,0,0,0,0
1,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah stretch silk camisole,Beige stretch silk Slips digit silk digit span...,Clothing Tops Tanks Camis,https://www.net-a-porter.com/us/en/product/119...,Fits true_size normal Cut slightly loose fit L...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,sizing,Regular,...,0,0,0,0,0,0,0,0,0,0
2,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah stretch silk camisole,Beige stretch silk Slips digit silk digit span...,Clothing Tops Tanks Camis,https://www.net-a-porter.com/us/en/product/119...,Fits true_size normal Cut slightly loose fit L...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,Weekend,...,0,0,0,0,0,0,0,0,0,0
3,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah stretch silk camisole,Beige stretch silk Slips digit silk digit span...,Clothing Tops Tanks Camis,https://www.net-a-porter.com/us/en/product/119...,Fits true_size normal Cut slightly loose fit L...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,Work,...,0,0,0,0,0,0,0,0,0,0
4,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah stretch silk camisole,Beige stretch silk Slips digit silk digit span...,Clothing Tops Tanks Camis,https://www.net-a-porter.com/us/en/product/119...,Fits true_size normal Cut slightly loose fit L...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,length_top,Mid-Length (at hips),...,0,0,0,0,0,0,0,0,0,0
